In [3]:
from evaluate import load
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
os.environ['MPLCONFIGDIR'] = '/nfs/nas-6.1/wclu/cache'

In [11]:
result = []
directory = '/nfs/nas-6.1/wclu/prompt_compression/output'
for root, dirs, files in os.walk(directory):
    for filename in tqdm(files):
        if 'random' in filename:
            df = pd.read_json(os.path.join(root, filename))
            config = filename.replace('random_','').replace('.json','').split('_')
            config.append(df)
            result.append(config)
df = pd.DataFrame(result, columns=['size','c_rate', 'df'])

  0%|          | 0/46 [00:00<?, ?it/s]

In [5]:
bertscore = load("bertscore")
def evaluate_bert(df):
    seq_predictions = df['seq_com_output'].tolist()
    para_predictions = df['para_com_output'].tolist()
    references = df['output'].tolist()
    seq_score = bertscore.compute(predictions=seq_predictions, references=references, model_type="microsoft/deberta-large-mnli", batch_size=16)
    para_score = bertscore.compute(predictions=para_predictions, references=references, model_type="microsoft/deberta-large-mnli", batch_size=16)
    seq_f1 = np.mean(seq_score['f1'])
    para_f1 = np.mean(para_score['f1'])
    return format(seq_f1,'.4f'), format(para_f1,'.4f')


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wu43cdje because the default path (/nfs/nas-6.1/wclu/cache) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [12]:
seq_bertscore = []
para_bertscore = []
for i in tqdm(range(len(df))):
    bert_score = evaluate_bert(df['df'][i])
    seq_bertscore.append(bert_score[0])
    para_bertscore.append(bert_score[1])

  0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
df['seq_bertscore'] = seq_bertscore
df['para_bertscore'] = para_bertscore

In [8]:
perplexity = load("perplexity",module_type="metric")
def evaluate_perplexity(df, size):
    #references = ('User: '+df['prompt']+'\nAssistant: '+df['output']).tolist()
    seq_predictions = (df['prompt']+' '+df['seq_com_output']).tolist()
    para_predictions = (df['prompt']+' '+df['para_com_output']).tolist()
    #ref_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=references, batch_size=16)
    seq_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=seq_predictions, batch_size=32)
    para_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=para_predictions, batch_size=32)
    #return format(ref_ppl['mean_perplexity'],'.3f'), format(seq_ppl['mean_perplexity'],'.3f'), format(para_ppl['mean_perplexity'],'.3f')
    return format(seq_ppl['mean_perplexity'],'.3f'), format(para_ppl['mean_perplexity'],'.3f')
    


In [9]:
seq_perplexity = []
para_perplexity = []
for i in tqdm(range(len(df))):
    ppl = evaluate_perplexity(df['df'][i], df['size'][i])
    seq_perplexity.append(ppl[0])
    para_perplexity.append(ppl[1])

  0%|          | 0/7 [00:00<?, ?it/s]

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
DEBUG: Possible options found for libcudart.so: {PosixPath('/usr/local/cuda/lib64/libcudart.so')}
CUDA SETUP: PyTorch settings found: CUDA_VERSION=117, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading binary /home/wclu/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...
libcusparse.so.11: cannot open shared object file: No such file or director

/home/wclu/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/home/wclu/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /home/wclu/anaconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
df['seq_ppl'] = seq_perplexity
df['para_ppl'] = para_perplexity

In [ ]:
df[df['size']=='13b'].sort_values('c_rate')

In [14]:
df[df['size']=='7b'].sort_values('c_rate')

,size,c_rate,df,seq_bertscore,para_bertscore
0,7b,0.1,...,0.6091,0.7375
1,7b,0.2,...,0.5789,0.6254
6,7b,0.3,...,0.5555,0.5709
2,7b,0.4,...,0.5344,0.5358
3,7b,0.5,...,0.5093,0.5214
4,7b,0.6,...,0.4815,0.5049
5,7b,0.7,...,0.4616,0.4666


## GPT

In [ ]:
references = ('A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions.\nUSER: ' + df['df'][11]['prompt']+'\nASSISTANT: '+df['df'][11]['seq_com_output']).tolist()
#references = ('USER: ' + df['df'][11]['prompt']+'\nASSISTANT: '+df['df'][11]['seq_com_output']).tolist()
print(references[0])

In [ ]:
ref_ppl = perplexity.compute(model_id=f'lmsys/vicuna-7b-v1.5', predictions=references, batch_size=32)
print(format(ref_ppl['mean_perplexity'],'.3f'))


In [ ]:
score = []

In [ ]:
for i in tqdm(range(len(score),5)):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
            "role": "system",
            "content": "Imagine that you have a super-intelligent AI assistant, and that you require help with the following question. Which answer best satisfies your needs?"
            },
            {
            "role": "user",
            "content": f"Question: {prompt_list[i]}\n\nAnswer G: <{references[i]}>\n\nAnswer F: <{seq_predictions[i]}>\n\nComparing these two answers, which answer is better?\n◼ Answer F is significantly better.\n◼ Answer U is significantly better.\n◼ Neither is significantly better."
            }
        ],
        temperature=0,
        max_tokens=30,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    score.append(response.choices[0].message.content)